In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import nibabel as nib
import os
from tensorflow.keras import Input, Sequential, Model
from tensorflow.keras.layers import Dense, Conv3D, MaxPooling3D, BatchNormalization, Dropout, concatenate, Flatten
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from pathlib import Path
from DataGenerator import DataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

2021-07-24 18:16:27.784621: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [73]:
import shutil
keras.backend.clear_session()

In [74]:
kfold = KFold(n_splits=5, shuffle=True)
alz = pd.read_csv("alzheimers.csv").drop_duplicates()
conv = pd.read_csv("converted.csv").drop_duplicates()
non = pd.read_csv("nonconverted.csv").drop_duplicates()
cont = pd.read_csv("control.csv").drop_duplicates()

for index, row in alz.iterrows():
    if not Path("petregistered/" + str(row["PET_ID"]) + "_registered.nii.gz").exists():
        alz = alz.drop(index)


for index, row in cont.iterrows():
    if not Path("petregistered/" + str(row["PET_ID"]) + "_registered.nii.gz").exists():
        cont = cont.drop(index)
        
for index, row in alz.iterrows():
    if not Path("registered/" + str(row["MRI_ID"]) + "_registered.nii.gz").exists():
        alz = alz.drop(index)


for index, row in cont.iterrows():
    if not Path("registered/" + str(row["MRI_ID"]) + "_registered.nii.gz").exists():
        cont = cont.drop(index)        
        
posmarker = np.ones(len(alz))
negmarker = np.zeros(len(cont))

alz["marker"] = posmarker
cont["marker"] = negmarker
subjects = alz.append(cont)
ids = subjects[["PET_ID", "MRI_ID"]]
markers = subjects["marker"]


In [75]:
acc_per_fold = []
loss_per_fold = []

fold_no = 1
for train, test in kfold.split(ids, markers):
    
    petinput = Input(shape = (182, 218, 182, 1))
    mriinput = Input(shape = (182, 218, 182, 1))
    x = Conv3D(16, kernel_size = 3, activation = "relu", kernel_initializer='he_uniform', input_shape = (182, 218, 182, 1))(petinput)
    x = MaxPooling3D(pool_size = (2,2,2))(x)
    x = BatchNormalization(center=True, scale=True)(x)
    x = Dropout(0.2)(x)
    x = Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)
    x = BatchNormalization(center=True, scale=True)(x)
    x = Dropout(0.2)(x)
    x = Model(inputs = petinput, outputs = x)

    y = Conv3D(16, kernel_size = 3, activation = "relu", kernel_initializer='he_uniform', input_shape = (182, 218, 182, 1))(mriinput)
    y = MaxPooling3D(pool_size = (2,2,2))(y)
    y = BatchNormalization(center=True, scale=True)(y)
    y = Dropout(0.2)(y)
    y = Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform')(y)
    y = MaxPooling3D(pool_size=(2, 2, 2))(y)
    y = BatchNormalization(center=True, scale=True)(y)
    y = Dropout(0.2)(y)
    y = Model(inputs = mriinput, outputs = y)

    end = concatenate([x.output, y.output])
    z = Flatten()(end)
    z = Dense(10, activation='relu', kernel_initializer='he_uniform')(z)
    z = Dense(8, activation='relu', kernel_initializer='he_uniform')(z)
    z = Dense(5, activation='relu')(z)
    z = Dense(1, activation = "sigmoid")(z)
    model = Model(inputs=[x.input, y.input], outputs=z)



# ids_train, ids_val, markers_train, markers_val = train_test_split(ids, markers, random_state = 42, stratify = markers, test_size = .25)

    model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["mae", "acc"])

#train_dataset = tf.data.Dataset.from_tensor_slices((ids_train, markers_train)).map(grabfile)
#val_dataset = tf.data.Dataset.from_tensor_slices((ids_val, markers_val))
    checkpoint_filepath = 'temp/checkpoint'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        save_freq = "epoch",
        monitor='val_acc',
        mode='max',
        save_best_only=True)

    batch_size = 2
    train_generator = DataGenerator(ids.iloc[train], markers.iloc[train], batch_size = batch_size)
    val_generator = DataGenerator(ids.iloc[test], markers.iloc[test], batch_size = batch_size)

    model.fit(train_generator, validation_data = val_generator, callbacks=[model_checkpoint_callback], epochs = 5)
    
    scores = model.evaluate(val_generator, verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    fold_no = fold_no + 1

Epoch 1/5
111/111 [==============================] - 3279s 30s/step - loss: 39.7884 - mae: 0.4553 - acc: 0.5495 - val_loss: 10.2700 - val_mae: 0.4821 - val_acc: 0.5179
Epoch 2/5
111/111 [==============================] - 3266s 29s/step - loss: 19.0416 - mae: 0.4128 - acc: 0.5856 - val_loss: 32.6769 - val_mae: 0.5179 - val_acc: 0.4821
Epoch 3/5
111/111 [==============================] - 3279s 30s/step - loss: 7.5990 - mae: 0.3536 - acc: 0.6396 - val_loss: 3.1713 - val_mae: 0.4268 - val_acc: 0.5536
Epoch 4/5
111/111 [==============================] - 3273s 29s/step - loss: 5.3929 - mae: 0.3080 - acc: 0.6982 - val_loss: 1.4981 - val_mae: 0.2395 - val_acc: 0.7857
Epoch 5/5
111/111 [==============================] - 3263s 29s/step - loss: 2.1811 - mae: 0.1795 - acc: 0.8153 - val_loss: 8.1743 - val_mae: 0.4821 - val_acc: 0.5179
Score for fold 1: loss of 8.174327850341797; mae of 48.21333289146423%
Epoch 1/5
111/111 [==============================] - 3282s 30s/step - loss: 22.7466 - mae: 0.50

In [67]:
model = keras.models.load_model('finishedmodels')

mode

In [39]:
for index, row in conv.iterrows():
    if not Path("petregistered/" + str(row["PET_ID"]) + "_registered.nii.gz").exists():
        conv = conv.drop(index)


for index, row in non.iterrows():
    if not Path("petregistered/" + str(row["PET_ID"]) + "_registered.nii.gz").exists():
        non = non.drop(index)
        
for index, row in conv.iterrows():
    if not Path("registered/" + str(row["MRI_ID"]) + "_registered.nii.gz").exists():
        conv = conv.drop(index)


for index, row in non.iterrows():
    if not Path("registered/" + str(row["MRI_ID"]) + "_registered.nii.gz").exists():
        non = non.drop(index)        
        
posmarker = np.ones(len(conv))
negmarker = np.zeros(len(non))

conv["marker"] = posmarker
non["marker"] = negmarker
subjects = conv.append(non)
ids = subjects[["PET_ID", "MRI_ID"]]
markers = subjects["marker"]

In [96]:
train_generator.__getitem__(0)

([[array([[[[0., 0., 0., ..., 0., 0., 0.],
            [0., 0., 0., ..., 0., 0., 0.],
            [0., 0., 0., ..., 0., 0., 0.],
            ...,
            [0., 0., 0., ..., 0., 0., 0.],
            [0., 0., 0., ..., 0., 0., 0.],
            [0., 0., 0., ..., 0., 0., 0.]],
   
           [[0., 0., 0., ..., 0., 0., 0.],
            [0., 0., 0., ..., 0., 0., 0.],
            [0., 0., 0., ..., 0., 0., 0.],
            ...,
            [0., 0., 0., ..., 0., 0., 0.],
            [0., 0., 0., ..., 0., 0., 0.],
            [0., 0., 0., ..., 0., 0., 0.]],
   
           [[0., 0., 0., ..., 0., 0., 0.],
            [0., 0., 0., ..., 0., 0., 0.],
            [0., 0., 0., ..., 0., 0., 0.],
            ...,
            [0., 0., 0., ..., 0., 0., 0.],
            [0., 0., 0., ..., 0., 0., 0.],
            [0., 0., 0., ..., 0., 0., 0.]],
   
           ...,
   
           [[0., 0., 0., ..., 0., 0., 0.],
            [0., 0., 0., ..., 0., 0., 0.],
            [0., 0., 0., ..., 0., 0., 0.],
           

In [91]:
y = np.empty((self.batch_size, 1), dtype=int)

        # Generate data
for i, label in enumerate(labels_temp):
    # Store sample
    y[i,] = label

NameError: name 'self' is not defined

In [56]:
fold_no = 1
for train, test in kfold.split(ids, markers):
    model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["mae", "acc"])

#train_dataset = tf.data.Dataset.from_tensor_slices((ids_train, markers_train)).map(grabfile)
#val_dataset = tf.data.Dataset.from_tensor_slices((ids_val, markers_val))
    checkpoint_filepath = 'temp/checkpoint'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        save_freq = "epoch",
        monitor='val_acc',
        mode='max',
        save_best_only=True)

    batch_size = 2
    train_generator = DataGenerator(ids.iloc[train], markers.iloc[train], batch_size = batch_size)
    val_generator = DataGenerator(ids.iloc[test], markers.iloc[test], batch_size = batch_size)

    model.fit(train_generator, validation_data = val_generator, callbacks=[model_checkpoint_callback], epochs = 5)
    
    scores = model.evaluate(val_generator, verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

  # Increase fold number
    fold_no = fold_no + 1

Epoch 1/5
64/64 [==============================] - 284s 4s/step - loss: 31835.4941 - mae: 0.4838 - acc: 0.5625 - val_loss: 17.1839 - val_mae: 0.4372 - val_acc: 0.5938
Epoch 2/5
64/64 [==============================] - 272s 4s/step - loss: 76.7541 - mae: 0.5242 - acc: 0.4922 - val_loss: 113.0965 - val_mae: 0.4847 - val_acc: 0.5000
Epoch 3/5
64/64 [==============================] - 272s 4s/step - loss: 62.7861 - mae: 0.5080 - acc: 0.4844 - val_loss: 29.0866 - val_mae: 0.4688 - val_acc: 0.5312
Epoch 4/5
64/64 [==============================] - 272s 4s/step - loss: 9780.2500 - mae: 0.4805 - acc: 0.5781 - val_loss: 15472.1963 - val_mae: 0.4998 - val_acc: 0.5000
Epoch 5/5
64/64 [==============================] - 272s 4s/step - loss: 159.8848 - mae: 0.5104 - acc: 0.4844 - val_loss: 12592.5078 - val_mae: 0.5000 - val_acc: 0.5000
Score for fold 1: loss of 12592.51171875; mae of 50.00040531158447%
Epoch 1/5
64/64 [==============================] - 284s 4s/step - loss: 11.2446 - mae: 0.4838 - acc

KeyboardInterrupt: 

In [60]:
print(model.summary())

Model: "functional_71"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           [(None, 182, 218, 18 0                                            
__________________________________________________________________________________________________
input_24 (InputLayer)           [(None, 182, 218, 18 0                                            
__________________________________________________________________________________________________
conv3d_45 (Conv3D)              (None, 180, 216, 180 448         input_23[0][0]                   
__________________________________________________________________________________________________
conv3d_47 (Conv3D)              (None, 180, 216, 180 448         input_24[0][0]                   
______________________________________________________________________________________

In [64]:
model.layers[22].trainable

True

In [65]:
model.evaluate(val_generator)

16/16 [==============================] - 48s 3s/step - loss: 0.6276 - mae: 0.4526 - acc: 0.5625


[0.627627432346344, 0.4525560140609741, 0.5625]

In [ ]:
model.load_weights('model.h5')

In [ ]:
img = nib.load("brains/322646.nii.gz")
img = img.get_fdata().squeeze()
print(img.shape)

In [ ]:
import matplotlib.pyplot as plt
def show_slices(slices):
    fig, axes = plt.subplots(1, len(slices))
    for i, slice in enumerate(slices):
        axes[i].imshow(slice.T, cmap="gray", origin="lower")

In [ ]:
slice_0 = img[90, :, :]
slice_1 = img[:, 100, :]
slice_2 = img[:, :, 160]

In [ ]:
show_slices([slice_0, slice_1, slice_2])


In [ ]:
for root, directories, files in os.walk("data"):
    for directory in directories:
        if "Mask" in directory:
            shutil.rmtree(root)


In [ ]:
PET = patients.loc[(patients["Modality"] == "PET")]["Image ID"].values


In [ ]:
patients = pd.read_csv("idaSearch_7_07_2021 (3).csv")


In [ ]:
347757 in PET

In [98]:
train_generator = DataGenerator(ids.iloc[train], markers.iloc[train], batch_size = batch_size)


In [112]:
for k in range(20):
    print(model(train_generator.__getitem__(k)[0]))

tf.Tensor(
[[0.9999995 ]
 [0.49368355]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.9999959 ]
 [0.49368355]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.99999666]
 [0.6620523 ]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.49368355]
 [0.6619325 ]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.999982  ]
 [0.49368355]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.9999965]
 [0.999979 ]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.49368355]
 [0.49368355]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.99997985]
 [0.49368355]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.99999404]
 [0.49368355]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.49368355]
 [0.99999714]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.49368355]
 [0.9999726 ]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.9999837]
 [1.       ]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.9999888]
 [0.9999947]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.9999978]
 [0.9999983]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.49368355]
 [0

In [113]:
for k in range(20):
    print(train_generator.__getitem__(k)[1])

[[1]
 [1]]
[[0]
 [0]]
[[0]
 [0]]
[[0]
 [0]]
[[1]
 [1]]
[[1]
 [0]]
[[1]
 [1]]
[[0]
 [0]]
[[0]
 [0]]
[[0]
 [0]]
[[1]
 [0]]
[[0]
 [1]]
[[0]
 [1]]
[[1]
 [0]]
[[0]
 [1]]
[[1]
 [0]]
[[1]
 [1]]
[[1]
 [0]]
[[1]
 [0]]
[[0]
 [1]]
